In [1]:
!pip install flwr flwr_datasets datasets torcheval -q
# !pip install datasets -q
!pip install flwr[simulation]@git+https://github.com/0x365/flower-fl.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.6/478.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not c

In [2]:
#@title {vertical-output: true}

from pathlib import Path
my_file = Path("smart_dao")
if not my_file.is_dir():
    !git clone https://github.com/strath-ace/smart-dao smart_dao

!cp smart_dao/agriculture-federated-learning/centralised-unet/UNET.py .
!cp smart_dao/agriculture-federated-learning/centralised-unet/utils.py .
!cp smart_dao/agriculture-federated-learning/setup/params_default.yml params.yml

Cloning into 'smart_dao'...
remote: Enumerating objects: 4396, done.
remote: Counting objects: 100% (722/722), done.
remote: Compressing objects: 100% (467/467), done.
remote: Total 4396 (delta 274), reused 685 (delta 246), pack-reused 3674 (from 1)
Receiving objects: 100% (4396/4396), 48.96 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (1931/1931), done.


In [3]:
# import warnings
# warnings.filterwarnings('once')

import argparse
from collections import OrderedDict
from typing import Dict, Tuple, List
import time
import os
import yaml
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import Scalar

from datasets import Dataset, load_dataset
from datasets.utils.logging import disable_progress_bar
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner, LinearPartitioner
from flwr_datasets.utils import divide_dataset

from torch.utils.data import random_split

from utils import *
# from core import *
from UNET import UNet



import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import Dataset
from torcheval.metrics.functional import multiclass_f1_score
from torch.utils.data import DataLoader
import sklearn.metrics

from tifffile import imread

import numpy as np


In [4]:
parser = argparse.ArgumentParser(description="Flower Simulation with PyTorch")

file_location = "."
if not os.path.exists(file_location+"/model_save"):
    os.mkdir(file_location+"/model_save")
with open(file_location+"/params.yml", "r") as f:
    params_config = yaml.load(f, Loader=yaml.SafeLoader)
# print(params_config)

TARGET_NUM_CLIENTS = params_config["TARGET_NUM_CLIENTS"]
NUM_ROUNDS = params_config["AGR_NUM_ROUNDS"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:

def dice_loss(pred, target, smooth=1.):
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

    return loss.mean()

criterion = torch.nn.CrossEntropyLoss()


def train(model,dataloaders, optimizer, num_classes, use_scheduler=False, epochs=10):
    """
    Train model and store results
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    stored_data = {"epoch": {}}
    for i, epoch in enumerate(range(epochs)):
        description = 'Epoch {}/{}'.format(epoch+1, epochs)

        model.train()

        sum_loss = 0
        totaler = 0

        for j, data in enumerate(dataloaders):#tqdm(dataloaders,desc=description)):

            # inputs = data["image"][:,:-1]
            # labels = data["image"][:,-1]

            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            multi_scale = 0.5   # 0.8 works (0.5, 0.25 good) (0.1 best result so far)

            loss_ce = criterion(outputs, labels)

            outputs = F.softmax(outputs, dim=1)
            loss_d = dice_loss(outputs, F.one_hot(labels, num_classes=num_classes).permute(0, 3, 1, 2).float())

            loss = loss_ce * multi_scale + loss_d * (1-multi_scale)

            loss.backward()
            optimizer.step()
            # if use_scheduler:
            #     scheduler.step()

            sum_loss += loss.data.cpu().numpy() * inputs.size(0)
            totaler += inputs.size(0)

        updater = {str(epoch+1): {"loss": sum_loss, "epoch_loss": sum_loss/totaler, "time": 3}}
        stored_data["epoch"].update(updater)
        print("----- Loss:", updater[str(epoch+1)]["epoch_loss"], "-----")

    return model, stored_data

In [6]:
def test(model, dataloaders, num_classes, num_epochs=25, threshold=0):
    """
    Test model and return results
    Return: loss, accuracy

    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()

    sum_loss = 0
    # sum_accuracy = 0
    totaler = 0

    big_confusion = np.zeros((num_classes, num_classes))
    number_in_class = np.zeros(num_classes)
    number_in_pred = np.zeros(num_classes)

    for data in dataloaders:#tqdm(dataloaders,desc=description)):

        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        pred = model(inputs)

        multi_scale = 0.5   # 0.8 works (0.5, 0.25 good) (0.1 best result so far)

        loss_ce = criterion(pred, labels)

        pred = F.softmax(pred, dim=1)

        loss_d = dice_loss(pred, F.one_hot(labels, num_classes=num_classes).permute(0, 3, 1, 2).float())

        loss = loss_ce * multi_scale + loss_d * (1-multi_scale)

        sum_loss += loss.data.cpu().numpy() * inputs.size(0)

        def argmax_with_threshold(tensor, threshold, default_value):
            # Check if all values are below the threshold
            below_threshold = (tensor < threshold).all(dim=1)

            # Compute the argmax normally
            argmax_indices = torch.argmax(tensor, dim=1)

            # If all values are below the threshold, set to the default_value
            argmax_indices[below_threshold] = default_value

            return argmax_indices

        if threshold == 0:
            pred = torch.argmax(pred, dim=1)
        else:
            pred = argmax_with_threshold(pred, threshold, 0)

        # sum_accuracy += multiclass_f1_score(torch.flatten(pred), torch.flatten(labels), num_classes=num_classes, average="weighted").cpu().numpy() * inputs.size(0)

        pred_out = pred.data

        flat_pred = pred_out.cpu().numpy().flatten().astype(int)
        flat_label = labels.cpu().numpy().flatten().astype(int)

        totaler += inputs.size(0)

        big_confusion += sklearn.metrics.confusion_matrix(flat_label, flat_pred, labels=np.arange(num_classes))#, normalize="true")

        binners = np.bincount(flat_label)
        binners_pred = np.bincount(flat_pred)

        number_in_class[range(len(binners))] += binners
        number_in_pred[range(len(binners_pred))] += binners_pred

    loss = float(sum_loss)/float(totaler)
    # accuracy = float(sum_accuracy)/float(totaler)

    # for i in range(num_classes):
    #     if number_in_class[i] > 0:
    #         big_confusion[i] = big_confusion[i]/number_in_class[i]

    return loss, big_confusion, number_in_class, number_in_pred

In [7]:
#@title {vertical-output: true}

dataset = load_dataset("0x365/eo-crop-type-belgium", split="train")

ds = dataset.with_format("torch")

ds_split = ds.train_test_split(test_size=0.2)

# print(dataset_new)

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, num_classes):
        # self.data = data
        self.bands = ["B02","B03","B04","B05","B06","B07","B08","B11","B12"]
        self.num_classes = num_classes
        self.processed_data = []

        for sample in data:
            inputs = torch.stack([sample[k][0] for k in self.bands], dim=0).float() / 255
            labels = sample["label"][0].long()
            # print(labels.shape)
            # print(num_classes)
            labels[labels > self.num_classes-1] = 0
            # print(torch.unique(labels))
            # labels_one_hot = F.one_hot(labels, num_classes=self.num_classes).permute(2, 0, 1).float()
            self.processed_data.append((inputs, labels))
        print("Built dataset")

    def __getitem__(self, idx):
        # sample = self.processed_data[idx]
        # inputs = torch.stack([sample[k][0] for k in self.bands], dim=0).float() / 255
        # labels = sample["label"][0].long()
        # # print(labels.shape)
        # # print(num_classes)
        # labels[labels > self.num_classes-1] = 0

        # return (inputs, labels)
        return self.processed_data[idx]

    def __len__(self):
        return len(self.processed_data)

    # def unqiue(self):
    #     return torch.unqiue(self.data["label"])

# ds_train = CustomDataset(ds_split["train"])#, num_classes=11)
# ds_test = CustomDataset(ds_split["test"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.68k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/352M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/354M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def reformat_json(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy array to list
    elif isinstance(obj, list) or isinstance(obj, tuple):
        return [reformat_json(item) for item in obj]  # Recursively apply to list items
    elif isinstance(obj, dict):
        return {key: reformat_json(value) for key, value in obj.items()}  # Handle dicts too
    else:
        return obj

In [9]:
# Flower client, adapted from Pytorch quickstart example
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainset, valset, num_classes):
        self.trainset = CustomDataset(trainset, num_classes)
        self.valset = CustomDataset(valset, num_classes)

        # Instantiate model
        # num_classes =  11
        self.num_classes = num_classes
        self.model = UNet(num_classes)

        # Determine device
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)  # send model to device

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        set_params(self.model, parameters)

        # Read from config
        # batch, epochs = config["batch_size"], config["epochs"]

        # Construct dataloader
        # print((self.trainset))
        trainloader = DataLoader(self.trainset, batch_size=20, shuffle=True)

        # Define optimizer
        # optimizer = torch.optim.SGD(self.model.parameters(), lr=params_config["LE_lr"], momentum=params_config["LE_momentum"])
        # optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        # Train
        # print(trainloader)
        # print(next(iter(trainloader)))
        self.model, stored_data = train(self.model, trainloader, optimizer, self.num_classes)


        # Return local model and statistics
        return self.get_parameters({}), len(trainloader.dataset), {}

    def evaluate(self, parameters, config):
        set_params(self.model, parameters)

        # Construct dataloader
        valloader = DataLoader(self.valset, batch_size=10)
        # Was 2 - changed to 32

        # Evaluate
        loss, big_confusion, number_in_class, number_in_pred = test(self.model, valloader, self.num_classes)

        extra_info = {
            "big_confusion": big_confusion,
            "number_in_class": number_in_class,
            "number_in_pred": number_in_pred
        }

        # Return statistics
        return float(loss), len(valloader.dataset), {"confusion": np.array(big_confusion).flatten().tolist()}


def get_client_fn(dataset, num_classes):
    """Return a function to construct a client.

    The VirtualClientEngine will execute this function whenever a client is sampled by
    the strategy to participate.
    """

    def client_fn(cid: str) -> fl.client.Client:
        """Construct a FlowerClient with its own dataset partition."""

        client_dataset = dataset.load_partition(int(cid))

        client_dataset_splits = client_dataset.train_test_split(test_size=0.1, seed=42)

        trainset = client_dataset_splits["train"]
        valset = client_dataset_splits["test"]

        # Create and return client
        return FlowerClient(trainset, valset, num_classes).to_client()

    return client_fn


def fit_config(server_round: int) -> Dict[str, Scalar]:
    """Return a configuration with static batch size and (local) epochs."""
    config = {
        "epochs": params_config["CL_epochs"],  # Number of local epochs done by clients
        "batch_size": params_config["CL_batch_size"],  # Batch size to use by clients during fit()
    }
    return config


def set_params(model: torch.nn.ModuleList, params: List[fl.common.NDArrays]):
    """Set model weights from a list of NumPy ndarrays."""
    params_dict = zip(model.state_dict().keys(), params)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """Aggregation function for (federated) evaluation metrics, i.e. those returned by
    the client's evaluate() method."""
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


def get_evaluate_fn(
    centralized_testset: Dataset,
    num_classes,
):
    """Return an evaluation function for centralized evaluation."""

    def evaluate(
        server_round: int, parameters: fl.common.NDArrays, config: Dict[str, Scalar]
    ):
        """Use the entire CIFAR-10 test set for evaluation."""

        # Determine device
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        model = UNet(num_classes)
        set_params(model, parameters)
        model.to(device)

        # Apply transform to dataset
        testset = CustomDataset(centralized_testset, num_classes)#.with_transform(apply_transforms)

        # Disable tqdm for dataset preprocessing
        disable_progress_bar()

        testloader = DataLoader(testset, batch_size=10)#params_config["AGR_batch_size"])
        loss, big_confusion, number_in_class, number_in_pred = test(model, testloader, num_classes)

        print(reformat_json(big_confusion))
        print(reformat_json(number_in_class))
        print(reformat_json(number_in_pred))

        extra_info = {
            "big_confusion": reformat_json(big_confusion),
            "number_in_class": reformat_json(number_in_class),
            "number_in_pred": reformat_json(number_in_pred)
        }
        print((extra_info))

        return loss, {"confusion": np.array(big_confusion).flatten().tolist()}

    return evaluate



In [ ]:
#@title {vertical-output: true}

# Resources to be assigned to each virtual client
client_resources = {
    "num_cpus": 10,#params_config["NUM_CPU"],
    "num_gpus": 0.1,#params_config["NUM_GPU"],
}

save_location = os.path.join(file_location, "output")
if not os.path.exists(save_location):
    os.makedirs(save_location)

centralized_testset = ds_split["test"]


PARAM_partition_style = ["iid", "niid"]
PARAM_num_classes = [11, 21]
PARAM_num_clients = [2, 32]

!cp drive/MyDrive/ESA/new_results_federated/* output/.

###################
starter = 0
##################

aggregation_rounds = 20


for partition_style in PARAM_partition_style:
    for num_classes in PARAM_num_classes:
        for num_clients in PARAM_num_clients:

            file_name = save_location+"/result"
            file_name += "_iteration_"+str(starter)
            file_name += "_partition_style_"+str(partition_style)
            file_name += "_num_classes_"+str(num_classes)
            file_name += "_num_clients_"+str(num_clients)
            file_name += ".json"

            if os.path.exists(file_name):
                print("File already computed for "+file_name)
                continue

            print("##############")
            print("Run test for", file_name)
            print("##############")

            if partition_style == "iid":
                partitioner = IidPartitioner(num_clients)
            elif partition_style == "niid":
                partitioner = LinearPartitioner(num_partitions=num_clients)

            partitioner.dataset = ds_split["train"]

            # Configure the strategy
            strategy = fl.server.strategy.FedAvg(
                fraction_fit=params_config["AGR_fraction_fit"],  # Sample 10% of available clients for training
                fraction_evaluate=params_config["AGR_fraction_evaluate"],  # Sample 5% of available clients for evaluation
                min_available_clients=params_config["AGR_min_available_clients"],
                on_fit_config_fn=fit_config,
                evaluate_metrics_aggregation_fn=weighted_average,  # Aggregate federated metrics
                evaluate_fn=get_evaluate_fn(centralized_testset, num_classes),  # Global evaluation function
            )

            # ClientApp for Flower-Next
            client = fl.client.ClientApp(
                client_fn=get_client_fn(partitioner, num_classes),
            )

            # ServerApp for Flower-Next
            server = fl.server.ServerApp(
                config=fl.server.ServerConfig(num_rounds=aggregation_rounds),
                # config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
                strategy=strategy,
            )

            # Start simulation
            stuff = fl.simulation.start_simulation(
                client_fn=get_client_fn(partitioner, num_classes),
                num_clients=num_clients,#num_clients,
                client_resources=client_resources,
                config=fl.server.ServerConfig(num_rounds=aggregation_rounds),
                # config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
                strategy=strategy,
                # actor_kwargs={
                #     "on_actor_init_fn": disable_progress_bar  # disable tqdm on each actor/process spawning virtual clients
                # },
            )

            json_out = {
                "iteration": starter,
                "partition_style": partition_style,
                "num_classes": num_classes,
                "num_clients": num_clients,
                "result": stuff.repr_json()
            }
            save_json(file_name, json_out)
            !cp output/* drive/MyDrive/ESA/new_results_federated/.


cp: cannot stat 'drive/MyDrive/ESA/new_results_federated/*': No such file or directory


INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout


##############
Run test for ./output/result_iteration_0_partition_style_iid_num_classes_11_num_clients_2.json
##############


2024-11-04 17:00:30,579	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 26731914854.0, 'memory': 53463829710.0, 'node:__internal_head__': 1.0, 'accelerator_type:A100': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 10, 'num_gpus': 0.1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=1951) 2024-11-04 17:00:33.327258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1951) 2024-11-04 17:00:33.349613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] 

(ClientAppActor pid=1951) Built dataset
(ClientAppActor pid=1951) Built dataset


...

Built dataset


INFO :      fit progress: (4, 0.3758389254742751, {'confusion': [54475723.0, 1722547.0, 910394.0, 692654.0, 456034.0, 323.0, 167326.0, 0.0, 0.0, 0.0, 0.0, 647306.0, 8689967.0, 40958.0, 169896.0, 34357.0, 0.0, 9826.0, 0.0, 0.0, 0.0, 0.0, 147074.0, 39292.0, 3157297.0, 5724.0, 7619.0, 0.0, 34476.0, 0.0, 0.0, 0.0, 0.0, 174106.0, 117816.0, 5665.0, 2496032.0, 57201.0, 0.0, 1536.0, 0.0, 0.0, 0.0, 0.0, 111979.0, 27713.0, 9285.0, 23979.0, 1161290.0, 0.0, 781.0, 0.0, 0.0, 0.0, 0.0, 881105.0, 20804.0, 3438.0, 21295.0, 3154.0, 247.0, 934.0, 0.0, 0.0, 0.0, 0.0, 50648.0, 7707.0, 27623.0, 2225.0, 822.0, 0.0, 664770.0, 0.0, 0.0, 0.0, 0.0, 448084.0, 26072.0, 3809.0, 1639.0, 472.0, 0.0, 309.0, 764.0, 0.0, 0.0, 0.0, 221344.0, 30642.0, 1712.0, 3140.0, 419.0, 0.0, 166.0, 78.0, 0.0, 0.0, 0.0, 229034.0, 19926.0, 1776.0, 9475.0, 4687.0, 0.0, 1395.0, 0.0, 0.0, 0.0, 0.0, 152258.0, 64575.0, 2070.0, 5666.0, 1406.0, 0.0, 262.0, 0.0, 0.0, 0.0, 0.0]}, 1576.396838575)
INFO :      configure_evaluate: strategy sampled 

[[54475723.0, 1722547.0, 910394.0, 692654.0, 456034.0, 323.0, 167326.0, 0.0, 0.0, 0.0, 0.0], [647306.0, 8689967.0, 40958.0, 169896.0, 34357.0, 0.0, 9826.0, 0.0, 0.0, 0.0, 0.0], [147074.0, 39292.0, 3157297.0, 5724.0, 7619.0, 0.0, 34476.0, 0.0, 0.0, 0.0, 0.0], [174106.0, 117816.0, 5665.0, 2496032.0, 57201.0, 0.0, 1536.0, 0.0, 0.0, 0.0, 0.0], [111979.0, 27713.0, 9285.0, 23979.0, 1161290.0, 0.0, 781.0, 0.0, 0.0, 0.0, 0.0], [881105.0, 20804.0, 3438.0, 21295.0, 3154.0, 247.0, 934.0, 0.0, 0.0, 0.0, 0.0], [50648.0, 7707.0, 27623.0, 2225.0, 822.0, 0.0, 664770.0, 0.0, 0.0, 0.0, 0.0], [448084.0, 26072.0, 3809.0, 1639.0, 472.0, 0.0, 309.0, 764.0, 0.0, 0.0, 0.0], [221344.0, 30642.0, 1712.0, 3140.0, 419.0, 0.0, 166.0, 78.0, 0.0, 0.0, 0.0], [229034.0, 19926.0, 1776.0, 9475.0, 4687.0, 0.0, 1395.0, 0.0, 0.0, 0.0, 0.0], [152258.0, 64575.0, 2070.0, 5666.0, 1406.0, 0.0, 262.0, 0.0, 0.0, 0.0, 0.0]]
[58425001.0, 9592310.0, 3391482.0, 2852356.0, 1335027.0, 930977.0, 753795.0, 481149.0, 257501.0, 266293.0, 22

INFO :      aggregate_evaluate: received 0 results and 2 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=1951) Built dataset
(ClientAppActor pid=1951) Built dataset
(ClientAppActor pid=1951) ----- Loss: 0.4093224559400873 -----
(ClientAppActor pid=1951) ----- Loss: 0.378264692696658 -----
(ClientAppActor pid=1951) ----- Loss: 0.37196368736548413 -----
(ClientAppActor pid=1951) ----- Loss: 0.3724444694333262 -----
(ClientAppActor pid=1951) ----- Loss: 0.3670343217579907 -----
(ClientAppActor pid=1951) ----- Loss: 0.36740943269738463 -----
(ClientAppActor pid=1951) ----- Loss: 0.3740479250564646 -----
(ClientAppActor pid=1951) ----- Loss: 0.3666166130432614 -----
(ClientAppActor pid=1951) ----- Loss: 0.36003610038359224 -----
(ClientAppActor pid=1951) ----- Loss: 0.3714415212066806 -----
(ClientAppActor pid=1951) Built dataset
(ClientAppActor pid=1951) Built dataset
(ClientAppActor pid=1951) ----- Loss: 0.3997860425920787 -----
(ClientAppActor pid=1951) ----- Loss: 0.3786012722189661 -----
(ClientAppActor pid=1951) ----- Loss: 0.3769586039551556 -----
(ClientAppActor pid